In [1]:
import os
import cv2
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.preprocessing import normalize
from mtcnn.mtcnn import MTCNN
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
model = load_model('facenet_keras.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## loading Images of suprer star............

In [4]:
cd /home/sourabh/Documents/facenet/celeb_faces/data/train/ben_afflek

/home/sourabh/Documents/facenet/celeb_faces/data/train/ben_afflek


In [5]:
items = os.listdir()
len(items)

73

In [6]:
def find_face(frame):
    detector = MTCNN()
    result = detector.detect_faces(frame)
    x,y,w,h = result[0]['box']
    x1,y1 = x+w,y+h
    face = frame[y:y1,x:x1]
    return face

In [7]:
faces, name = [],[]
for i in items:

    try:
        # reading image as an array.
        frame = cv2.imread(i)
        # calling find_face() function.
        face = find_face(frame)
        # resizing into (160,160)
        face = cv2.resize(face,(160,160))
        # normalizing the faces data.
        face = (face-face.mean())/face.std()
        # appending face in list: faces.
        faces.append(face)

        if i.startswith('b'):
            name.append(0)
        if i.startswith('j'):
            name.append(1)
        if i.startswith('k'):
            name.append(2)
        if i.startswith('m'):
            name.append(3)
        print(i)
        
    except: pass

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
kaling16.jpg
madona8.jpg
jerry5.jpg
madona10.jpg
kaling17.jpg
kaling20.jpg
jerry11.jpg
kaling18.jpg
ben4.jpg
kaling2.jpg
kaling19.jpg
ben5.jpg
jerry20.jpg
kaling22.jpg
jerry8.jpg
kaling7.jpg
jerry9.jpg
jerry1.jpg
jerry13.jpg
kaling9.jpg
jerry18.jpg
ben3.jpg
jerry17.jpg
ben6.jpg
madona4.jpg
kaling11.jpg
madona12.jpg
jerry16.jpg
ben11.jpg
madona2.jpg
jerry7.jpg
madona15.jpg
kaling3.jpg
kaling15.jpg
kaling1.jpg
madona16.jpg
jerry19.jpg
ben7.jpg
jerry2.jpg
ben1.jpg
ben2.jpg
kaling12.jpg
jerry10.jpg
kaling4.jpg
madona9.jpg
madona3.jpg
madona7.jpg
madona19.jpg
madona5.jpg
madona14.jpg
madona11.jpg
ben8.jpg
ben12.jpg
kaling8.jpg
ben10.jpg
madona6.jpg
kaling6.jpg
jerry14.jpg
madona1.jpg
madona18.jpg
jerry4.jpg
ben13.jpg
kaling21.jpg
jerry3.jpg
jerry15.jpg
madona17.jpg
ben9.jpg
jerry6.jpg
kaling10.jpg
jerry12.jpg
kaling14.jpg
kaling5.jpg


In [8]:
# list of all the faces.
# all faces have diff dimensions.
faces = np.asarray(faces)

## facenet_model

In [9]:
ans=model.predict(np.expand_dims(faces[0],axis=0))

In [10]:
vector = []
for face in faces:
    predictions = model.predict(np.expand_dims(face,axis=0))
    vector.append(np.squeeze(predictions,axis=0))

In [11]:
vector = np.asarray(vector)

In [12]:
vector = normalize(vector,norm='l2')

In [13]:
vector.shape

(72, 128)

In [34]:
Y = np.zeros((72,4))
for i in range(72):
    Y[i][name[i]]+=1

## classification

In [28]:
clf = SVC(kernel = 'linear')

In [44]:
x_train,x_test,y_train,y_test = train_test_split(vector,name,test_size=0.2,random_state=1)

In [45]:
x_train.shape,y_train.shape

((57, 128), (57, 1))

In [46]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [47]:
pred = clf.predict(x_test)

In [48]:
clf.score(x_test,y_test)

0.9333333333333333